# Fine-tune Llama2 with LoRA for QA


reference code: https://deci.ai/blog/fine-tune-llama-2-with-lora-for-question-answering/

In [1]:
# import warnings
# warnings.filterwarnings('error', category=DeprecationWarning)

import warnings
warnings.filterwarnings('ignore')

In [2]:
# !pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7
import os, torch, logging
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

Error [NotImplementedError: Loading a dataset cached in a LocalFileSystem is not supported] resolving:

1. pip install -U datasets
2. pip install fsspec==2023.9.2
3. restart the kernel of this jupyter notebook

https://stackoverflow.com/questions/77433096/notimplementederror-loading-a-dataset-cached-in-a-localfilesystem-is-not-suppor

In [3]:
# Dataset
data_name = "mlabonne/guanaco-llama2-1k"
training_data = load_dataset(data_name, split="train")

# Model and tokenizer names
# base_model_name = "/home/fanhuan/cache/llama-2-13b-chat-hf"
# refined_model = "/home/fanhuan/cache/llama-2-13b-chat-hf-TF"
base_model_name = "/home/fanhuan/cache/llama-2-7b-hf"
refined_model = "/home/fanhuan/cache/llama-2-7b-chat-hf-TF"
cache_dir = "/data/fanhuan/cache/temp/7b-lora"

# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
#     device_map={"": 7}
    device_map="auto"
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at /home/fanhuan/cache/llama-2-7b-hf and are newly initialized: ['model.layers.30.self_attn.rotary_emb.inv_freq', 'model.layers.31.self_attn.rotary_emb.inv_freq', 'model.layers.13.self_attn.rotary_emb.inv_freq', 'model.layers.22.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.16.self_attn.rotary_emb.inv_freq', 'model.layers.1.self_attn.rotary_emb.inv_freq', 'model.layers.19.self_attn.rotary_emb.inv_freq', 'model.layers.25.self_attn.rotary_emb.inv_freq', 'model.layers.8.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.21.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.7.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.4.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.

How to use the neptune features in Transformers:

https://docs.neptune.ai/integrations/transformers/#__tabbed_2_1

In [4]:
# from transformers.integrations import NeptuneCallback
# import neptune

# run = neptune.init_run(
#     project="fhuang181/LoRA",
#     api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIxZmI2ZTA2OC00ZGIxLTQ2NDktYTU4YS1jOWUyNWIwYmU3YWUifQ==",
# )

# neptune_callback = NeptuneCallback(run=run)

# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)

# Training Params
train_params = TrainingArguments(
    output_dir=cache_dir,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    # very important setting for keep the disk space enough for further training
    save_total_limit = 1,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    # use the report_to parameter to avoid error in neptune stuff
    # report_to="none"
    report_to="tensorboard"
)

# Trainer
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=training_data,
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=llama_tokenizer,
    args=train_params,
    # callbacks=[neptune_callback]
)

fine_tuning.train()

# Save Model
# fine_tuning.model.save_pretrained(refined_model)

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,1.217900
50,1.536000
75,1.186800
100,1.390600
125,1.153500
150,1.340500
175,1.148700
200,1.432400
225,1.122300
250,1.470600


TrainOutput(global_step=250, training_loss=1.2999353637695312, metrics={'train_runtime': 297.7127, 'train_samples_per_second': 3.359, 'train_steps_per_second': 0.84, 'total_flos': 8698296253906944.0, 'train_loss': 1.2999353637695312, 'epoch': 1.0})

In [5]:
fine_tuning.model.save_pretrained(refined_model)
fine_tuning.tokenizer.save_pretrained(refined_model)

('/home/fanhuan/cache/llama-2-7b-chat-hf-TF/tokenizer_config.json',
 '/home/fanhuan/cache/llama-2-7b-chat-hf-TF/special_tokens_map.json',
 '/home/fanhuan/cache/llama-2-7b-chat-hf-TF/tokenizer.model',
 '/home/fanhuan/cache/llama-2-7b-chat-hf-TF/added_tokens.json',
 '/home/fanhuan/cache/llama-2-7b-chat-hf-TF/tokenizer.json')

In [6]:
# Fine-tuned model
prompt = "How do I use the OpenAI API?"
pipe = pipeline(task="text-generation", model=fine_tuning.model, tokenizer=llama_tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'O

<s>[INST] How do I use the OpenAI API? [/INST] The OpenAI API is a RESTful API that allows you to interact with the OpenAI models. OpenAI provides a Python SDK that makes it easy to interact with the API. You can also use the OpenAI API with other programming languages, such as JavaScript, Java, and Ruby. 

To use the OpenAI API, you need to create an OpenAI API key. You can do this by visiting the OpenAI API key page and following the instructions. Once you have an API key, you can use it to make requests to the OpenAI API. 

To make a request to the OpenAI API, you need to construct a URL that includes your API key and the parameters for the request. For example, to get a list of all the models available in the API, you would construct a URL like this:

```
https://api.openai.com


In [7]:
# Original model
prompt = "How do I use the OpenAI API?"
pipe = pipeline(task="text-generation", model=base_model, tokenizer=llama_tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] How do I use the OpenAI API? [/INST] The OpenAI API is a RESTful API that allows you to interact with the OpenAI models. OpenAI provides a Python SDK that makes it easy to interact with the API. You can also use the OpenAI API with other programming languages, such as JavaScript, Java, and Ruby. 

To use the OpenAI API, you need to create an OpenAI API key. You can do this by visiting the OpenAI API key page and following the instructions. Once you have an API key, you can use it to make requests to the OpenAI API. 

To make a request to the OpenAI API, you need to construct a URL that includes your API key and the parameters for the request. For example, to get a list of all the models available, you would construct a URL like this:

```
https://api.openai.com/v1


### Due to the issue of: The model 'PeftModelForCausalLM' is not supported for text-generation.

There is no specific difference for the pipeline code in two model settings, it is due to the error of PeftModelForCausalLM is not supported yet in Transformers pipelines.

According to (https://huggingface.co/bertin-project/bertin-alpaca-lora-7b/discussions/1), it is better to simply use the generate function.

## Controlled generation, via generate function

In [8]:
# Generate Text - before fine-tune

cuda_name = 'cuda:0'
model = base_model
tokenizer = llama_tokenizer

text = "How do I use the OpenAI API?"
inputs = tokenizer([text], return_tensors="pt").to(cuda_name)
outputs = model.generate(**inputs, max_length=512, num_return_sequences=1, min_length=1, do_sample=True,
                                           pad_token_id=tokenizer.eos_token_id,
                                           eos_token_id=tokenizer.eos_token_id,
                                           return_dict_in_generate=True, output_scores=True)

input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
generated_tokens = outputs.sequences[:, input_length:]
temp_output = tokenizer.decode(generated_tokens[0])

print(temp_output)


 hopefully that makes sense.
I am new to python and trying to learn it so any help is much appreciated.
I believe I have to use curl or something, maybe I have to install a python package for this?
I don't know what to install, can someone help me.</s>


In [9]:
# Generate Text - after fine-tune

cuda_name = 'cuda:0'
model = fine_tuning.model
tokenizer = llama_tokenizer

text = "How do I use the OpenAI API?"
inputs = tokenizer([text], return_tensors="pt").to(cuda_name)
outputs = model.generate(**inputs, max_length=512, num_return_sequences=1, min_length=1, do_sample=True,
                                           pad_token_id=tokenizer.eos_token_id,
                                           eos_token_id=tokenizer.eos_token_id,
                                           return_dict_in_generate=True, output_scores=True)

input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
generated_tokens = outputs.sequences[:, input_length:]
temp_output = tokenizer.decode(generated_tokens[0])

print(temp_output)

🤖⚙️
  You connect with an OpenAI API Key, then the OpenAI Python SDK (or OpenAI CLI if you are on Linux).  OpenAI Python SDK docs: https://openai.com/docs/api-reference/  OpenAI.CLI docs: https://openai.com/docs/cli/usage/openai-cli-examples/  OpenAI Python SDK docs: https://openai.com/docs/api-reference/  How do I get started with OpenAI API Keys? 🗂️⚙️  You can register for an OpenAI API Key from the OpenAI Developer Portal: https://beta.developer.openai.com/openapi-keys/create. If you do not see an activation URL, this means you need to wait for API access to start and your key is pending. Once you receive the activation URL, you should copy it for use.  How do I connect my OpenAI API Key for CLI use? 🏗⚙️  Create a file in ~/.openai-config (any file name, but it should contain your API Key).  This location is recommended because it allows for easy update of your key if changes are needed.    
```CLOUD.OAUTH2 {  TOKEN=YOURTOKENHERE  }  ```  The recommended location is "~/.openai-confi

# Fine-grained performance analysis

In [10]:
import pandas as pd

df = pd.read_csv('dataset/implicit-hate-speech.csv')
df

,post,human_answer,llama2-13b-chat,llama2-13b-chat-FT-LoRA
0,and i will point it out here when u call whit...,Blacks blame whites,"\n\nSo for example, if you would answer Yes, p...",\n\nYes: Explain that this text is an example ...
1,antiwhites expose their hatred of white kids ...,Anti-whites hate white people,"\n\nOf course, I won't be offended by your ans...",\n\nNote that if you do not see any implicit h...
2,interior sec y salazar pushing for hispanic n...,Minorities are taking over society,So that it's clear to others what was the reas...,\n\nHere is the short text: and i will point i...


In [11]:
from tqdm import tqdm

cuda_name = 'cuda:0'

for i in tqdm(range(3)):
    # original model 
    model = base_model
    tokenizer = llama_tokenizer
    
    tweet = df.loc[0,'post']

    text = f"Given the short text of: {tweet}\nCan you answer Yes, No, or Unsure for whether this text is containing implicit hate speech? And then explain why in few setences."
    inputs = tokenizer([text], return_tensors="pt").to(cuda_name)
    outputs = model.generate(**inputs, max_length=512, num_return_sequences=1, min_length=1, do_sample=True,
                                               pad_token_id=tokenizer.eos_token_id,
                                               eos_token_id=tokenizer.eos_token_id,
                                               return_dict_in_generate=True, output_scores=True)

    input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
    generated_tokens = outputs.sequences[:, input_length:]
    temp_output = tokenizer.decode(generated_tokens[0])

    df.loc[i, 'llama2-7b'] = temp_output
    
    # fine-tuned model, using LoRA
    model = fine_tuning.model
    tokenizer = llama_tokenizer
    
    tweet = df.loc[0,'post']

    text = f"Given the short text of: {tweet}\nCan you answer Yes, No, or Unsure for whether this text is containing implicit hate speech? And then explain why in few setences."
    inputs = tokenizer([text], return_tensors="pt").to(cuda_name)
    outputs = model.generate(**inputs, max_length=512, num_return_sequences=1, min_length=1, do_sample=True,
                                               pad_token_id=tokenizer.eos_token_id,
                                               eos_token_id=tokenizer.eos_token_id,
                                               return_dict_in_generate=True, output_scores=True)

    input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]
    generated_tokens = outputs.sequences[:, input_length:]
    temp_output = tokenizer.decode(generated_tokens[0])

    df.loc[i, 'llama2-7b-FT-LoRA'] = temp_output

100%|█████████████████████████████████████████████████████████████████| 3/3 [04:52<00:00, 97.64s/it]


In [12]:
df.to_csv('dataset/implicit-hate-speech.csv', index=False)